In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [ ]:
!pip install -U tensorflow==2.15.0
import tensorflow as tf
tf.__version__

In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow==2.15.0

In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install --upgrade tf-models-official

In [ ]:
folder_path = "/kaggle/input/2024-sum-dpl-302-m/devset_images/devset_images"
label_path = "/kaggle/input/2024-sum-dpl-302-m/devset_images_gt.csv"


In [ ]:
import pandas as pd 
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
pip install -U scikit-learn

In [ ]:

data = pd.read_csv(label_path) 
# Preview the first 5 lines of the loaded data 
data_array=data.values
data_array[0:10]

In [ ]:
X_train=[]
Y_train=[]
k=0
for i in data_array:
    image = cv2.imread(folder_path+'/'+str(i[0])+'.jpg')
    if image is not None:
        #print(k)
        k=k+1
        image = cv2.resize(image, (224,224))
        # print(image.shape)
        X_train.append(image)
        Y_train.append(i[1])
print(k)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,10))
for i in range(1,11):
    plt.subplot(2,5,i)
    plt.grid(False)
    plt.imshow(X_train[i])
    plt.xlabel("Label " + str(Y_train[i]))
plt.show()

In [ ]:
X_train=np.array(X_train)
Y_train= np.array(Y_train)
print(X_train.shape)
print(Y_train.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train)

In [ ]:
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

# Giả định rằng X_train và Y_train đã được định nghĩa trước đó
# và X_train chứa các ảnh có giá trị trong khoảng [0, 1] hoặc [0, 255]

plt.figure(figsize=(16, 10))
for i in range(1, 11):
    plt.subplot(2, 5, i)
    plt.grid(False)
    
    # Lấy một ảnh từ X_train và thêm một chiều batch
    image = tf.expand_dims(X_train[i], 0)
    
    # Thực hiện augmentation trên ảnh
    augmented_image = img_augmentation(image)
    
    # Loại bỏ chiều batch
    augmented_image = tf.squeeze(augmented_image, axis=0)
    
    # Chuyển đổi ảnh về dạng numpy array
    augmented_image = augmented_image.numpy()
    
    # Nếu ảnh đã được chuẩn hóa về khoảng [0, 1], chuyển đổi lại về [0, 255]
    if augmented_image.max() <= 1.0:
        augmented_image = augmented_image * 255.0
    
    # Đảm bảo ảnh có kiểu dữ liệu uint8
    augmented_image = augmented_image.astype('uint8')
    
    # Hiển thị ảnh
    plt.imshow(augmented_image)
    plt.xlabel("Label " + str(Y_train[i]))
    
plt.show()


In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
IMG_SIZE = 224
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB4,ResNet101V2
from tensorflow.keras.applications.efficientnet import preprocess_input
#from tensorflow.keras.applications.resnet_v2 import preprocess_input
def build_model(num_classes=1):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = preprocess_input(inputs)
    x = img_augmentation(x)
    model = EfficientNetB4(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation="relu", name="relu")(x)

    top_dropout_rate = 0.2

    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)

    x = layers.Dense(128, activation="relu", name="relu_2")(x)
    #change 
    x = layers.Dropout(0.5, name="top_dropout_2")(x)

    outputs = layers.Dense(num_classes, activation="sigmoid", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy',f1]
    )
    return model

In [ ]:
model = build_model(num_classes=1)

In [ ]:
import tensorflow as tf

def outer_product(x):
    #Einstein Notation  [batch,1,1,depth] x [batch,1,1,depth] -> [batch,depth,depth]
    phi_I = tf.einsum('ijkm,ijkn->imn',x[0],x[1])
    # Reshape from [batch_size,depth,depth] to [batch_size, depth*depth]
    phi_I = tf.reshape(phi_I,[-1,x[0].shape[3]*x[1].shape[3]])
    
    # Divide by feature map size [sizexsize]
    size1 = int(x[1].shape[1])
    size2 = int(x[1].shape[2])
    phi_I = tf.divide(phi_I, size1*size2)
    
    # Take signed square root of phi_I
    y_ssqrt = tf.multiply(tf.sign(phi_I),tf.sqrt(tf.abs(phi_I)+1e-12))
    
    # Apply l2 normalization
    z_l2 = tf.nn.l2_normalize(y_ssqrt, axis=1)
    return z_l2

In [ ]:
from keras.models import Model
from tensorflow.keras.layers import Convolution2D,Activation,GlobalAveragePooling2D,MaxPooling2D,Flatten,Dense,Dropout,Input,Reshape,Lambda


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = preprocess_input(inputs)
bandau = img_augmentation(x)
# Model 1 

model1 = EfficientNetB4(include_top=False, input_tensor=bandau, weights="imagenet")

# Model 2
model2 = EfficientNetB4(include_top=False, input_tensor=bandau, weights="imagenet")

for i, layer in enumerate(model1.layers):
    layer._name = 'model1_' + layer.name
    layer.trainable = False #Freeze all layers
for i, layer in enumerate(model2.layers):
    layer._name = 'model2_' + layer.name
    layer.trainable = False #Freeze all layers


# Rebuild top
#x = layers.GlobalAveragePooling2D(name="avg_pool")(model1.output)
last_output1 = layers.BatchNormalization()(model1.output)

# Rebuild top
#x = layers.GlobalAveragePooling2D(name="avg_pool")(model2.output)
last_output2 = layers.BatchNormalization()(model2.output)

##
model1_ = Model(inputs=model1.input, outputs=last_output1)
model2_ = Model(inputs=model2.input, outputs=last_output2)

d1=model1_.output
d2=model2_.output

bilinear = Lambda(outer_product)([d1,d2])

x = Dense(32, activation='relu', name='dense1')(bilinear)

x = layers.Dropout(0.2, name="top_dropout")(x)

predictions=Dense(1, activation='sigmoid', name='predictions')(x)

model = Model(inputs=model1.input, outputs=predictions)

In [ ]:
# Compile
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(
    optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy',f1]
)

model.summary()


In [ ]:
import tensorflow as tf

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='./dgt301-{epoch:03d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.weights.h5', 
    monitor='val_loss',  
    save_weights_only=True, 
    save_freq='epoch', 
    verbose=1,
    save_best_only=True,
    mode='auto'  # This can be 'auto', 'min', or 'max'
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0,
    patience=10,
    verbose=1
)

reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=6,
    verbose=1,
    min_delta=0.001,  # `epsilon` has been replaced with `min_delta`
    cooldown=0,
    min_lr=0.00001
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

# Precision calculation
def precision(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# Recall calculation
def recall(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    possible_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# F1 score calculation
def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

# Compile the model with custom metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1])

# Callbacks
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='./dgt301-{epoch:03d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.weights.h5', 
    monitor='val_loss',  
    save_weights_only=True, 
    save_freq='epoch', 
    verbose=1,
    save_best_only=True,
    mode='auto'
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0,
    patience=10,
    verbose=1
)

reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=6,
    verbose=1,
    min_delta=0.001,
    cooldown=0,
    min_lr=0.00001
)

# Fit the model
history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=[model_checkpoint, early_stopping, reduce_learning_rate]
)


In [ ]:
model.load_weights('./dgt301-003_loss-0.6633_val_loss-0.6514.weights.h5')

def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-400:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy",f1]
    )

unfreeze_model(model)

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(mode='auto', filepath='./dgt301_turn2-{epoch:03d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.weights.h5', 
                     monitor='val_loss',  
                     save_weights_only='True', 
                     period=1,
                     verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=10,
                               verbose=1)
reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.2,
                                         patience=6,
                                         verbose=1,
                                         epsilon=0.001,
                                         cooldown=0,
                                         min_lr=0.00001)

In [ ]:
import tensorflow as tf

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    mode='auto',
    filepath='./dgt301_turn2-{epoch:03d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.weights.h5', 
    monitor='val_loss',  
    save_weights_only=True,  # Corrected to boolean True
    save_freq='epoch',  # Save weights at the end of each epoch
    verbose=1
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0,
    patience=10,
    verbose=1
)

reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=6,
    verbose=1,
    min_delta=0.001,
    cooldown=0,
    min_lr=0.00001
)


In [ ]:
history2 = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=[model_checkpoint,early_stopping,reduce_learning_rate]
)

In [ ]:
label_file = "/kaggle/input/2024-sum-dpl-302-m/test.csv"
folder_path = "/kaggle/input/2024-sum-dpl-302-m/testset_images/testset_images"

In [ ]:
data = pd.read_csv(label_file) 
data_array=data.values

In [ ]:
data.head()

In [ ]:
X_train=[]
k=0
for i in data_array:
  image = cv2.imread(folder_path+'/'+str(i[0])+'.jpg')
  if image is not None:
    k=k+1
    image = cv2.resize(image, (224,224))
    # print(image.shape)
    X_train.append(image)
  else:
    pass

In [ ]:
X_train=np.array(X_train)
X_train.shape

In [ ]:
model.load_weights('./dgt301_turn2-012_loss-0.0300_val_loss-0.2798.weights.h5')

In [ ]:
batch_size = 32 

stacked = 0
#Y=np.append(Y_1, Y_2, axis=0)

for i in range(len(X_train)):
    if i % batch_size == 0:
        if stacked != 0: # First batch
            if stacked == 1:
                y = model(X_train[i-32:i])
                Y = np.array(y)
            else:
                y = model(X_train[i-32:i])
                Y = np.append(Y,y,axis=0)
            print(i-32,i,Y.shape)       
        stacked += 1
rest_output = model(X_train[(stacked-1)*32:])

Y = np.append(Y,rest_output,axis=0)

print(Y.shape)

In [ ]:
Y = np.array(Y)

In [ ]:
import csv

with open('submission_B4_bestloss_0.98.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    employee_writer.writerow(['Id'])
    j = 0
    cnt = 0
    for i in data_array:
        image = cv2.imread(folder_path + '/' + str(i[0]) + '.jpg')
        cnt += 1
        if image is not None:
            if Y[j] > 0.98:
                employee_writer.writerow([i[0]])
            else:
                pass
            j += 1
        else:
            pass
    print(cnt)


In [ ]:
import numpy as np
import pandas as pd
import cv2
import csv

# Đường dẫn đến tệp nhãn
label_file = "/kaggle/input/2024-sum-dpl-302-m/test.csv"

# Thư mục chứa hình ảnh kiểm tra
folder_path = "/kaggle/input/2024-sum-dpl-302-m/testset_images/testset_images"

# Đọc dữ liệu từ tệp nhãn
data = pd.read_csv(label_file)
data_array = data.values

# Đọc và xử lý hình ảnh từ thư mục
X_test = []
for i in data_array:
    image = cv2.imread(folder_path + '/' + str(i[0]) + '.jpg')
    if image is not None:
        image = cv2.resize(image, (224, 224))
        X_test.append(image)

# Chuyển danh sách hình ảnh thành mảng numpy
X_test = np.array(X_test)

# Tải trọng số của mô hình đã được huấn luyện trước
model.load_weights('./dgt301_turn2-012_loss-0.0300_val_loss-0.2798.weights.h5')

# Dự đoán
Y_pred = model.predict(X_test)

# Ghi kết quả vào tệp CSV
with open('test.csv', mode='w') as file:
    writer = csv.writer(file)
    writer.writerow(['image_id', 'label'])  # Viết tiêu đề cột
    for i, prediction in enumerate(Y_pred):
        if prediction > 0.99:  # Kiểm tra ngưỡng
            writer.writerow([data_array[i][0], 1])  # Ghi Id của hình ảnh và label dự đoán vào tệp
        else:
            writer.writerow([data_array[i][0], 0])  # Ghi Id của hình ảnh và label dự đoán vào tệp
